In [1]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



In [2]:
df_jan = pd.read_parquet('../Data/yellow_tripdata_2023-01.parquet')   
df_jan.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


### Number of columns

In [3]:
print(len(df_jan.columns))

19


### Compute duration in minutes:



In [4]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60
print(df_jan.duration.std())

42.59435124195458


### Dropping Outliers – Fraction of Records Left

In [5]:
df_filtered = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)].copy()
fraction_remaining = len(df_filtered) /len(df_jan)
print(fraction_remaining * 100)

98.1220282212598


### One-hot Encoding – Dimensionality of Feature Matrix

In [6]:
categorical = ['PULocationID', 'DOLocationID']  
df_filtered[categorical] = df_filtered[categorical].astype(str)

train_dicts = df_filtered[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print(X_train.shape[1])


515


### Train Linear Regression – RMSE on Train

In [7]:
y_train = df_filtered['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(rmse)

7.649261946248764


### Evaluate Model on February Data – RMSE on Validation

In [8]:
df_feb = pd.read_parquet('../Data/yellow_tripdata_2023-02.parquet')

# Compute duration
df_feb['duration'] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime).dt.total_seconds() / 60

# Filter outliers
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)].copy()

# Apply same transformations
df_feb[categorical] = df_feb[categorical].astype(str)
val_dicts = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_feb['duration'].values

# Predict and evaluate
y_pred_val = lr.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(rmse_val)

7.811817544074326
